In [2]:
import sys
from pathlib import Path
from datetime import datetime

# Añade src al path
sys.path.append(str(Path().resolve().parent / "src"))

In [3]:
# Configura el entorno de trabajo
%reload_ext autoreload
%autoreload 2

In [4]:
from data import load_raw_data_v2
import pandas as pd

# Descargar datos desde 2024 hasta la actualidad

# Con esto vamos a hacer un backfill de los datos de años anteriores
# y así tener un dataset completo para el año actual
# y el año anterior

from_year = 2024
to_year = datetime.now().year
print(f'Downloading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year+1):
    
    # download data for the whole year
    rides_one_year = load_raw_data_v2(year)
    
    # append rows
    rides = pd.concat([rides, rides_one_year])

rides.describe()


File 2024-01 was already in local storage
File 2024-02 was already in local storage
File 2024-03 was already in local storage
File 2024-04 was already in local storage
File 2024-05 was already in local storage
File 2024-06 was already in local storage
File 2024-07 was already in local storage
File 2024-08 was already in local storage
File 2024-09 was already in local storage
File 2024-10 was already in local storage
File 2024-11 was already in local storage
File 2024-12 was already in local storage
File 2025-01 was already in local storage
File 2025-02 was already in local storage
2025-03 file is not available
2025-04 file is not available
2025-05 file is not available
2025-06 file is not available
2025-07 file is not available
2025-08 file is not available
2025-09 file is not available
2025-10 file is not available
2025-11 file is not available
2025-12 file is not available


,pickup_datetime,pickup_location_id
count,48222016,4.822202e+07
mean,2024-08-06 01:50:30.125478,1.642402e+02
min,2024-01-01 00:00:00,1.000000e+00
25%,2024-04-21 13:46:25.750000,1.320000e+02
50%,2024-08-07 16:25:55,1.610000e+02
75%,2024-11-19 15:55:48.250000,2.330000e+02
max,2025-02-28 23:59:59,2.650000e+02
std,NaN,6.445180e+01


In [5]:
# Tenemos un total de 48M filas
print(f'{len(rides)} rows downloaded')

48222016 rows downloaded


In [6]:
from data import transform_to_time_series

# Transformamos a time series con la función transform_to_time_series
ts_data = transform_to_time_series(rides)

Rellenando huecos por localización: 100%|██████████| 263/263 [00:00<00:00, 296.98it/s]


In [ ]:
import os
from dotenv import load_dotenv
from paths import PARENT_DIR
import hopsworks

# Logeamos en hopsworks

load_dotenv(PARENT_DIR / '.env')


# Nombre del proyecto en hopsworks
HOPSWORKS_PROJECT_NAME =  'taxo_demand'

# API Key de hopsworks
HOPSWORKS_API_KEY = "9JzCqysXLMaEbsBJ"

project = hopsworks.login(
    api_key_value=HOPSWORKS_API_KEY,

)

2025-05-08 13:54:20,313 INFO: Initializing external client
2025-05-08 13:54:20,314 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-08 13:54:21,834 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1224869


In [8]:
# Creamos el feature store
feature_store = project.get_feature_store()

# Definimos el nombre y version del feature group
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

In [9]:
# Creamos el feature group
# Desactivamos el stream de Kafka
feature_group = feature_store.create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Hourly time series data",
    primary_key=['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
    stream=False,    # desactiva completamente Kafka
)

In [ ]:
# Guardamos el feature group
feature_group.insert(
    ts_data,
    write_options={"wait_for_job": True}
)